In [ ]:
# 웹툰탭 XPATH //*[@id="menu"]/li[2]/a
# DICT -> LIST -> DICT구성
# 월 //*[@id="wrap"]/header/div[3]/nav/ul/li[2]/a
# 화 //*[@id="wrap"]/header/div[3]/nav/ul/li[3]/a
# 수 //*[@id="wrap"]/header/div[3]/nav/ul/li[4]/a
# 들고오는 형태
# webToonList = [
#     {
#         "dayOfTheWeek": "월",
#             "webtoon": [
#                 {
#                     "title": "참교육",
#                     "auther": "채용택 / 한가람",
#                     "grade": 9.87,
#                     "postImage": "https://asdfsdafsdf.jpg"
#                 },
#                 {
#                     "title": "퀘스트지상주의",
#                     "auther": "박태준 만화회사",
#                     "grade": 8.00,
#                     "postImage": "https://postaddress.jpg"
#                 }
#             ]
#     }
# ]

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


driver = webdriver.Chrome(executable_path="chromedriver")

driver.get("https://comic.naver.com/index")
time.sleep(3)

driver.find_element(by=By.XPATH, value='//*[@id="menu"]/li[2]/a').click()
time.sleep(2)
webToonList = []
for i in range(2, 9):
    driver.find_element(by=By.XPATH, value=f'//*[@id="wrap"]/header/div[3]/nav/ul/li[{i}]/a').click()
    time.sleep(1)

    scrollTop = 0
    while True:
        driver.execute_script(f'window.scrollTo(0, {scrollTop})')

        scrollTop += 500
        time.sleep(0.5)
        if scrollTop > driver.execute_script("return document.body.scrollHeight"):
            break

    day = driver.find_element(by=By.XPATH, value=f'//*[@id="wrap"]/header/div[3]/nav/ul/li[{i}]/a').text
    webToon = []
    items = driver.find_elements(by=By.CSS_SELECTOR, value='.ContentList__content_list--q5KXY .item')
    for item in items:
        title = item.find_element(by=By.CSS_SELECTOR, value='.ContentTitle__title--e3qXt .text').text
        auther = item.find_element(by=By.CSS_SELECTOR, value='.ContentAuthor__author--CTAAP').text
        grade = item.find_element(by=By.CSS_SELECTOR, value='.rating_area .text').text
        postImage = item.find_element(by=By.CSS_SELECTOR, value='.Poster__link--sopnC .Poster__image--d9XTI')
        image = postImage.get_attribute(name="src")
        webToonDict = {
            "title": title,
            "auther": auther,
            "grade": grade,
            "postImage": image
        }
        webToon.append(webToonDict)
    dayDict = {
        "dayOfTheWeek": day,
        "webtoon": webToon
    }

    webToonList.append(dayDict)

webToonList


In [ ]:
import pandas as pd

df = pd.DataFrame(webToonList)
print(df)

In [ ]:
# 강사님 답
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(executable_path="chromedriver")

# 네이버 웹툰 메인화면 로딩
driver.get("https://comic.naver.com/index")

time.sleep(1)
# 상단 웹툰 탭으로 이동
webtoon_menu = driver.find_element(by=By.XPATH, value='//*[@id="menu"]/li[2]/a')
webtoon_menu.click()

webtoon_list = []
day_list = ["월", "화", "수", "목", "금", "토", "일"]

for day in day_list:
    dayOfTheWeek_menus = driver.find_elements(by=By.CSS_SELECTOR, value='.SubNavigationBar__link--PXX5B')
    for dayOfTheWeek_menu in dayOfTheWeek_menus:
        if(dayOfTheWeek_menu.text != day):
            continue

        dayOfTheWeek_menu.click()

        time.sleep(1)
        # 스크롤을 내려야 웹툰 표지를 들고옴
        scrollTop = 0
        while True:
            # scrollTo(x축, y축): x축, y축으로 이동해라
            driver.execute_script(f"window.scrollTo(0, {scrollTop});")
            # 한번 움직일 때 움직이는 단위
            scrollTop += 500
            # 너무 빠르게 내랴가면 정보를 못불러와서 속도를 조절해줌
            time.sleep(0.5)
            # scrollTop이 전체 높이보다 커지면 break(정지해라)
            if scrollTop > driver.execute_script("return document.body.scrollHeight"):
                break

        time.sleep(0.5)

        webtoon_items = driver.find_elements(by=By.CSS_SELECTOR, value='.ContentList__content_list--q5KXY .item')
        webtoon_items_list = []
        for item in webtoon_items:
            title = item.find_element(by=By.CSS_SELECTOR, value='.ContentTitle__title--e3qXt').text
            author = item.find_element(by=By.CSS_SELECTOR, value='.ContentAuthor__author--CTAAP').text
            rating = item.find_element(by=By.CSS_SELECTOR, value='.Rating__star_area--dFzsb > .text').text
            img = item.find_element(by=By.CSS_SELECTOR, value='.Poster__image--d9XTI').get_attribute(name="src")
            
            item_dict = {
                "title": title,
                "author": author,
                "rating": rating,
                "img": img
            }
            webtoon_items_list.append(item_dict)

        webtoon_by_day_dict = {
            "day": day,
            "webtoon_items_list": webtoon_items_list
        }

        webtoon_list.append(webtoon_by_day_dict)

df = pd.DataFrame(webtoon_list)
print(df)

In [ ]:
# 엑셀로 뽑을때는 아래와 같은 방식으로 자료가 있어야함
new_list = []

for webtoon in webtoon_list:
    for webtoon_item in webtoon.get("webtoon_items_list"):
        new_dict = {
            "day": webtoon.get("day")
        }
        new_dict.update(webtoon_item)
        new_list.append(new_dict)

df = pd.DataFrame(new_list)
print(df)
# 엑셀로 저장(to_excel은 encoding 필요 없음, index없애기 가능)
# df.to_csv("webtoon.csv", encoding="cp949")
df.to_excel("webtoon.xlsx", index=False)

In [ ]:
# json형태로 자료 변환
import json

new_list = []

with open("webtton.json", "w", encoding="utf-8") as f:
    json.dump(webtoon_list, f, indent=4, ensure_ascii=False)